In [26]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb


indicators = pd.read_csv('Indicators.csv')
stock = pd.read_csv('stocks-us-adjClose.csv')
sector = pd.read_csv('Company_information.csv', encoding = "ISO-8859-1")

In [27]:
sector.head()



,Symbol,Company.Name,Sector,Industry
0,ED,"Consolidated Edison, Inc.",Utilities,Electric Utilities
1,DD,E. I. du Pont de Nemours and Company,Basic Materials,Agricultural Chemicals
2,CVX,Chevron Corporation,Basic Materials,Major Integrated Oil & Gas
3,FL,"Foot Locker, Inc.",Consumer Goods,Textile - Apparel Footwear & Accessories
4,CAT,Caterpillar Inc.,Industrial Goods,Farm & Construction Machinery


In [28]:
s = sector.loc('Sector')

KeyError: <pandas.core.indexing._LocIndexer object at 0x10f9d2b00>

,Symbol,Company.Name,Sector,Industry
0,ED,"Consolidated Edison, Inc.",Utilities,Electric Utilities
1,DD,E. I. du Pont de Nemours and Company,Basic Materials,Agricultural Chemicals
2,CVX,Chevron Corporation,Basic Materials,Major Integrated Oil & Gas
3,FL,"Foot Locker, Inc.",Consumer Goods,Textile - Apparel Footwear & Accessories
4,CAT,Caterpillar Inc.,Industrial Goods,Farm & Construction Machinery
5,IP,International Paper Company,Consumer Goods,Packaging & Containers
6,SJW,SJW Group,Utilities,Water Utilities
7,F,Ford Motor Company,Consumer Goods,Auto Manufacturers - Major
8,LLY,Eli Lilly and Company,Healthcare,Drug Manufacturers - Major
9,AVP,"Avon Products, Inc.",Consumer Goods,Personal Products
